In [1]:
%run Dataset_cleanup/cleanup_clinical.py

Clinical data set imported


 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 



 Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'histologicaltype'
 'numberoflymphnodes' 'pathologyTstage' 'radiationtherapy' 'residualtumor'
 'vitalstatus'] 


Variables that are known at the time of diagnosis:
 ['dateofinitialpathologicdiagnosis' 'gleasonscore' 'psavalue' 'race'
 'yearstobirth']

Dimensions of clinical dataframe: (499, 5)


In [6]:
print(clinical.dtypes,"\n\n")
print(clinical.head())
print("\n\n",clinical.describe())
print("\n\n",clinical['race'].describe())

dateofinitialpathologicdiagnosis    float64
gleasonscore                        float64
psavalue                            float64
race                                 object
yearstobirth                        float64
dtype: object 


                dateofinitialpathologicdiagnosis  gleasonscore  psavalue  \
clinical_index                                                             
TCGA-CH-5753                              2008.0           9.0      4.53   
TCGA-CH-5763                              2008.0           7.0      0.10   
TCGA-CH-5764                              2008.0           7.0     13.95   
TCGA-CH-5772                              2008.0           9.0      0.03   
TCGA-CH-5790                              2006.0           7.0      0.10   

                 race  yearstobirth  
clinical_index                       
TCGA-CH-5753    white          70.0  
TCGA-CH-5763    white          66.0  
TCGA-CH-5764    white          66.0  
TCGA-CH-5772    white          63.0  
TC

/Users/ccthomps/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [ ]:
%run Dataset_cleanup/cleanup_gene_counts.py

## Analysis of initial clinical presentation for  predictors of metastasis

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced',
                            presort=False)

clf.fit(clinical, y)
print(clf.tree_)